In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [3]:
df = pd.read_csv('ga_final.csv')
df['date_format'] = pd.to_datetime(df['date'], format='%Y%m%d')
df['start_time'] = pd.to_datetime(df['visitStartTime'], unit='s')

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/758175610.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ga_final.csv')


In [4]:
# df.to_csv('df.csv', index=False)

In [5]:
date_df = df[['fullVisitorId', 'date_format']]
date_df['year'] = date_df['date_format'].dt.strftime('%Y')
date_df['month'] = date_df['date_format'].dt.strftime('%m')
date_df['week'] = date_df['date_format'].dt.isocalendar().week.astype(str)
date_df['year_month'] = date_df['year'] + '-' + date_df['month']
date_df['year_week'] = date_df['year'] + '-' + date_df['week']
date_df['dau'] = date_df.groupby('date_format')['fullVisitorId'].transform('nunique')
date_df['wau'] = date_df.groupby('year_week')['fullVisitorId'].transform('nunique')
date_df['mau'] = date_df.groupby('year_month')['fullVisitorId'].transform('nunique')
date_df = date_df.drop(columns=['fullVisitorId'])
date_df = date_df.drop_duplicates()
date_df = date_df.sort_values(by=['date_format'])

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1201203262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['year'] = date_df['date_format'].dt.strftime('%Y')
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1201203262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['month'] = date_df['date_format'].dt.strftime('%m')
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1201203262.py:4: SettingWithCopyWarning: 
A value is trying to be set on

In [6]:
# date_df.to_csv('date_df.csv', index=False)

In [7]:
original = df[['date_format', 'fullVisitorId']]
temp = original['date_format'].dt.year.astype(str) + '-' + original['date_format'].dt.month.astype(str)

def get_semester(series):
    if series in ['2016-8', '2016-9']:
        return 1
    elif series in ['2016-10', '2016-11', '2016-12']:
        return 2
    elif series in ['2017-1', '2017-2', '2017-3']:
        return 3
    elif series in ['2017-4', '2017-5', '2017-6']:
        return 4
    elif series in ['2017-7', '2017-8']:
        return 5
        
temp = pd.DataFrame(temp.apply(get_semester))
    # temp.groupby['date_format']['fullVisitorId'].apply(pd.DataFrame.nunique)
original = pd.concat([original, temp], axis=1)
original.columns = ['date_format', 'id', 'semester']

customers_in_one = original[(original['semester'] == 1)]['id'].unique()
customers_in_two = original[(original['semester'] == 2)]['id'].unique()
customers_in_three = original[(original['semester'] == 3)]['id'].unique()
customers_in_four = original[(original['semester'] == 4)]['id'].unique()
customers_in_five = original[(original['semester'] == 5)]['id'].unique()

    # 1분기 신규
one_new_user = len(customers_in_one)
    # 2분기 신규
two_new_user = original[original['semester'] == 2][~original['id'].isin(customers_in_one)]['id'].nunique()
    # 3분기 신규
three_new_user = original[original['semester'] == 3][~original['id'].isin(customers_in_one) &\
                                                        ~original['id'].isin(customers_in_two)]['id'].nunique()
    # 4분기 신규
four_new_user = original[original['semester'] == 4][~original['id'].isin(customers_in_one) &\
                                                        ~original['id'].isin(customers_in_two) &\
                                                        ~original['id'].isin(customers_in_three)]['id'].nunique()
    # 5분기 신규
five_new_user = original[original['semester'] == 5][~original['id'].isin(customers_in_one) &\
                                                        ~original['id'].isin(customers_in_two) &\
                                                        ~original['id'].isin(customers_in_three) &\
                                                        ~original['id'].isin(customers_in_four)]['id'].nunique()

    # 1분기 이탈
one_churn_user = original[original['semester'] == 1][~original['id'].isin(customers_in_two) &\
                                                        ~original['id'].isin(customers_in_three) &\
                                                        ~original['id'].isin(customers_in_four) &\
                                                        ~original['id'].isin(customers_in_five)]['id'].nunique()
    # 2분기 이탈
two_churn_user = original[original['semester'] == 2][~original['id'].isin(customers_in_three) &\
                                                        ~original['id'].isin(customers_in_four) &\
                                                        ~original['id'].isin(customers_in_five)]['id'].nunique()
    # 3분기 이탈
three_churn_user = original[original['semester'] == 3][~original['id'].isin(customers_in_four) &\
                                                        ~original['id'].isin(customers_in_five)]['id'].nunique()
    # 4분기 이탈
four_churn_user = original[original['semester'] == 4][~original['id'].isin(customers_in_five)]['id'].nunique()

    # 5분기 이탈 없음

    # 각 분기 유저수 
sau_one = original[(original['semester'] == 1)]['id'].nunique()
sau_two = original[(original['semester'] == 2)]['id'].nunique()
sau_three = original[(original['semester'] == 3)]['id'].nunique()
sau_four = original[(original['semester'] == 4)]['id'].nunique()

    # 각 분기 이탈률
one_churn_rate = round(one_churn_user / sau_one, 2)
two_churn_rate = round(two_churn_user / sau_two, 2)
three_churn_rate = round(three_churn_user / sau_three, 2)
four_churn_rate = round(four_churn_user / sau_four, 2)

    # 각 분기 cc 
x_third_cc = one_new_user / one_churn_rate
x_fourth_cc = two_new_user / two_churn_rate
n_first_cc = three_new_user / three_churn_rate
n_second_cc = four_new_user / four_churn_rate

cc = {}
cc['2016 삼분기'] = x_third_cc
cc['2016 사분기'] = x_fourth_cc
cc['2017 일분기'] = n_first_cc
cc['2017 이분기'] = n_second_cc
temp2 = pd.Series(cc)

sau = {}

sau['2016 삼분기'] = sau_one
sau['2016 사분기'] = sau_two
sau['2017 일분기'] = sau_three
sau['2017 이분기'] = sau_four
temp = pd.Series(sau)

res = pd.concat([temp2, temp], axis=1).reset_index()
res.columns = ['quarter', 'carrying capacity', 'qau']

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/2075497521.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  two_new_user = original[original['semester'] == 2][~original['id'].isin(customers_in_one)]['id'].nunique()
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/2075497521.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  three_new_user = original[original['semester'] == 3][~original['id'].isin(customers_in_one) &\
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/2075497521.py:35: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  four_new_user = original[original['semester'] == 4][~original['id'].isin(customers_in_one) &\
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/2075497521.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  five_new_user = original[original['semester'] == 5][~origina

In [8]:
# res.to_csv('res.csv', index=False)

In [9]:
from datetime import datetime
train_df2 = df[['fullVisitorId', 'date_format', 'start_time']]
train_df2.columns = ['fullVisitorId', 'date', 'start_time']
group = train_df2.groupby(['date', 'fullVisitorId'])['start_time']

result = pd.concat([group.min(), group.max()], axis=1)
result.columns = ['min', 'max']
result['min'] = result['min'].dt.strftime('%H:%M:%S')
result['max'] = result['max'].dt.strftime('%H:%M:%S')
    
result2 = result[result['max'] > result['min']]
result2.reset_index(inplace=True)

result2['min2'] = result2['min'].apply(lambda x: datetime.strptime(str(x), '%H:%M:%S'))
result2['max2'] = result2['max'].apply(lambda x: datetime.strptime(str(x), '%H:%M:%S'))
result2['max_min_minus'] = result2['max2'] - result2['min2']
result2['max_min_minus'] = result2['max_min_minus'] / pd.Timedelta(1, 'h')
id_max_min_minus_mean = pd.DataFrame(result2.groupby('fullVisitorId')['max_min_minus'].mean())
total_max_min_minus_mean = result2['max_min_minus'].sum() / 47217

ecdf = id_max_min_minus_mean / total_max_min_minus_mean
ecdf.reset_index(inplace=True)

# ecdf.to_csv('ecdf.csv', index=False)

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1647924287.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2['min2'] = result2['min'].apply(lambda x: datetime.strptime(str(x), '%H:%M:%S'))
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1647924287.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result2['max2'] = result2['max'].apply(lambda x: datetime.strptime(str(x), '%H:%M:%S'))
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1647924287.py

In [10]:
week_df = df[['fullVisitorId', 'date_format', 'start_time']]
week_df['year'] = week_df['date_format'].dt.isocalendar().year
week_df['week'] = week_df['date_format'].dt.isocalendar().week
week_df['current_week'] = week_df['year'] * 52 + week_df['week']
index_lst = week_df['year'] + (week_df['week'] / 100)
index_lst = sorted(list(index_lst.unique()))
week_df['start_week'] = week_df.groupby(['fullVisitorId'])[['current_week']].transform('min')
week_df['c_ind'] = week_df['current_week'] - week_df['start_week']
week_df['start_week'] = week_df['start_week'] - week_df['start_week'].min()
grouping = week_df.groupby(['start_week', 'c_ind'])
cohort_data = grouping['fullVisitorId'].apply(pd.Series.nunique)
cohort_data = pd.DataFrame(cohort_data)
cohort_data = cohort_data.reset_index()
cohort_counts = cohort_data.pivot(index='start_week', columns='c_ind', values='fullVisitorId')
retention = cohort_counts
indices = cohort_counts.index.to_list()
column_inds = cohort_counts.columns.to_list()
for i in range(len(indices)):
    m = indices[i]
    for j in range(len(column_inds)-1, 0, -1):
        retention.at[(m, column_inds[j])] = retention.at[(m, column_inds[j])] / retention.at[(m, column_inds[j-1])]
    retention.at[(m, 0)] = retention.at[(m, 0)] / retention.at[(m, 0)]
    
# retention.to_csv('retention.csv', index=False)    

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/3404824880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_df['year'] = week_df['date_format'].dt.isocalendar().year
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/3404824880.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_df['week'] = week_df['date_format'].dt.isocalendar().week


In [11]:
# 전체 52주
start_week = '2016-08-01'
end_week = '2017-07-31'
num_of_weeks = int((pd.to_datetime(end_week) - pd.to_datetime(start_week)) / np.timedelta64(1, 'W'))
hourday = df[['fullVisitorId', 'date_format', 'start_time']]
hourday['day'] = hourday['date_format'].dt.day_name()
hourday['time'] = hourday['start_time'].dt.hour
    
to_replace = [i for i in range(24)]
value = []
for i in range(24):
    if i == 0:
        value.append(f'12 am')
    elif 1 <= i < 12:
        value.append(f'{i} am')
    elif i == 12:
        value.append(f'12 pm')
    else:
        value.append(f'{i-12} pm')
            
hourday['time'].replace(to_replace=to_replace, value=value, inplace=True)
hourday = hourday[(hourday['date_format'] >= start_week) & (hourday['date_format'] <= end_week)]
grouping = hourday.groupby(['day', 'time'])
week_data = grouping['fullVisitorId'].apply(pd.Series.nunique)
week_data = pd.DataFrame(week_data).divide(num_of_weeks).round(1)
    
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
week_data = week_data.reset_index()
week_count = week_data.pivot(index='time', columns='day', values='fullVisitorId')
week_count = week_count.reindex(columns=order)
week_count = week_count.reindex(value)

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1325184305.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hourday['day'] = hourday['date_format'].dt.day_name()
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1325184305.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hourday['time'] = hourday['start_time'].dt.hour
/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_20625/1325184305.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [12]:
# week_count.to_csv('week_count.csv', index=False)

In [13]:
date_channel = df[['date_format', 'channelGrouping', 'fullVisitorId']]
date_channel.columns = ['date', 'channelGrouping', 'fullVisitorId']
year_month = date_channel['date'].dt.year.astype(str) + '-' + date_channel['date'].dt.month.astype(str)

year_month = pd.DataFrame(year_month)
date_channel = pd.concat([date_channel, year_month], axis=1)

date_channel.columns = ['date', 'channelGrouping', 'fullVisitorId', 'monthly']
mon_chan = pd.DataFrame(date_channel.groupby(['monthly', 'channelGrouping'])['fullVisitorId'].count())
channel_1610 = mon_chan.loc['2016-10',].reset_index()
channel_1611 = mon_chan.loc['2016-11',].reset_index()
mon_chan = mon_chan.reset_index()

In [15]:
mon_chan.to_csv('mon_chan.csv', index=False)
channel_1610.to_csv('1610.csv', index=False)
channel_1611.to_csv('1611.csv', index=False)